# Scalable Agent Collaboration for E-commerce: Building a Flexible Fulfillment Pipeline with Modular LLM-Powered Agents and Dynamic Tracking

In [77]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = YOUR_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [85]:
import uuid
import asyncio
from typing import List, Dict, Any

import adk
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.apps.app import App
from google.adk.agents import LlmAgent
from google.adk.tools.function_tool import FunctionTool
from google.adk.models.google_llm import Gemini
from google.genai import types

In [102]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

model_config = Gemini(
    model="gemini-2.5-flash-lite",
    http_options={"retry": retry_config}
)

# Creating a mock database 
MOCK_DB = {
    "inventory": {"item-A7Z": 5, "item-B2Y": 0},
    "orders": {}, # Changed to dict for easier tracking number lookup
    "tracking_statuses": { # Simulate dynamic tracking statuses
        "TRK-ABCDEF": "In Transit - Expected Delivery: Tomorrow",
        "TRK-0A7B9B": "Out for Delivery - Arriving Today!",
        "TRK-XYZ123": "Delivered on 2023-10-26"
    }
}

In [104]:
# DEFINING THE SPECIALIZED AGENTS
# ---- A. Inventory Specialist ----
def check_inventory_func(item_id: str) -> str:
    """Checks if item is in stock, Returns AVAILABLE OR OUT_OF_STOCK."""
    print(f"\n[Inventory Agent] Checking stock for {item_id}...")
    stock = MOCK_DB["inventory"].get(item_id, 0)
    return "AVAILABLE" if stock > 0 else "OUT_OF_STOCK"  
    
inventory_specialist = LlmAgent(
    name = "agent_inventory",
    model=model_config,
    instruction="You are an Inventory Specialist. Use your tools to check the stock levels.",
    tools=[FunctionTool(check_inventory_func)]
)

# ---- B. Shipping Specialist ----
def ship_item_func(item_id: str, address: str) -> str:
    """Ships an item to an address. Returns a tracking number."""
    print(f"\n[Shipping Agent] Shipping {item_id} to {address}...")
    return f"TRK-{uuid.uuid4().hex[:6].upper()}"

shipping_specialist = LlmAgent(
    name="agent_shipping",
    model=model_config,
    instruction="You are a Shipping Specialist. Use your tool to ship items and provide tracking number.",
    tools=[FunctionTool(ship_item_func)]
)

# ---- C. Database Specialist ----
def save_order_func(item_id: str, tracking_number: str) -> str:
    """Saves the completed order to the database."""
    print(f"\n[Databse Agent] Saving record: {item_id} -> {tracking_number}")
    MOCK_DB["orders"][tracking_number] = {"item": item_id, "tracking": tracking_number, "status": "Order Saved"}
    return "SUCCESS"
    
db_specialist = LlmAgent(
    name="agent_db",
    model=model_config,
    instruction="You are a Database Specialist. Use your tool to save the final order details into the database.",
    tools=[FunctionTool(save_order_func)]
)

# --- D.  nkTracking Agent ---
def get_tracking_status_func(tracking_number: str) -> str:
    """Retrieves the current tracking status for a given tracking number."""
    print(f"\n[Tracking] Looking up status for {tracking_number}...")
    status = MOCK_DB["tracking_statuses"].get(tracking_number, "Tracking number not found.")
    return status

tracking_specialist = LlmAgent(
    name="agent_tracking",
    model=model_config,
    tools=[FunctionTool(get_tracking_status_func)],
)

In [105]:
# Helper Functions
# --- Helper to extract text from an Event's content parts ---
def get_event_text(event: Any) -> str:
    if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts') and event.content.parts:
        # Concatenate text from all parts if multiple are present
        return "".join([part.text for part in event.content.parts if hasattr(part, 'text') and part.text])
    return ""

# --- Manager ---
def create_agent_runner_tool(agent: LlmAgent, tool_name: str, description: str):
    tool_session_service = InMemorySessionService()
    tool_runner = Runner(
        agent=agent,
        app_name=f"{agent.name}-app",
        session_service=tool_session_service
    )
    
    async def run_other_agent(prompt: str, session_id: str) -> str:
        """
        Runs the specified specialist agent with the given prompt in a given session.
        {description}
        """
        print(f"\n[Manager -> {agent.name}] Delegating: '{prompt}'")

        events = await tool_runner.run_debug(
            user_messages=prompt,
            user_id="default-user",
            session_id=session_id
        )

        final_output = ""
        for event in events:
            event_text = get_event_text(event)
            if event_text:
                final_output = event_text

            if hasattr(event, 'content') and event.content and \
               hasattr(event.content, 'parts') and event.content.parts:
                for part in event.content.parts:
                    if hasattr(part, 'function_response') and part.function_response and \
                       hasattr(part.function_response, 'response') and \
                       'result' in part.function_response.response:
                        function_result = part.function_response.response['result']
                        print(f"[{agent.name} -> Manager] Function Result: {function_result}")
                        return str(function_result) 

        return final_output

    docstring_template = run_other_agent.__doc__ if run_other_agent.__doc__ else ""
    run_other_agent.__doc__ = docstring_template.format(description=description)
    run_other_agent.__name__ = tool_name
    return FunctionTool(run_other_agent)

tool_delegate_inventory = create_agent_runner_tool(
    inventory_specialist,
    "check_inventory_via_agent",
    "Use this to ask the Inventory Specialist Agent about item availability. Provide the item ID."
)

tool_delegate_shipping = create_agent_runner_tool(
    shipping_specialist,
    "ship_item_via_agent",
    "Use this to ask the Shipping Specialist Agent to ship an item. Provide the item ID and address."
)

tool_delegate_db = create_agent_runner_tool(
    db_specialist,
    "save_order_via_agent",
    "Use this to ask the Database Specialist Agent to save order details. Provide the item ID and tracking number."
)

tool_delegate_tracking = create_agent_runner_tool(
    tracking_specialist, "get_tracking_status_via_agent",
    "Ask Tracking Specialist for an order's status. Provide the tracking number and a session ID."
)


# ---- MANAGER AGENT ----
manager_agent = LlmAgent(
    name="agent_manager",
    model=model_config,
    instruction="""
        You are a Fulfillment Manager supervising a team of specialists.
        \nYOUR GOAL: Process user orders and provide tracking information by delegating tasks to your team using the provided tools.
        Each tool corresponds to delegating a task to a specialist agent.
        When calling a tool, you must provide a `session_id` (e.g., a unique identifier like 'order-123') along with the task-specific information.
        \n\nPROCESS ORDER (STEPS 1-4):
        1. To check inventory, use `check_inventory_via_agent`. Provide the item ID and a `session_id`.
        2. IF the inventory agent reports 'OUT_OF_STOCK', apologize to the user and STOP.
        3. IF the inventory agent reports 'AVAILABLE', use `ship_item_via_agent`. Provide the item ID, address, and the same `session_id`. CAPTURE THE TRACKING NUMBER RETURNED.
        4. Use `save_order_via_agent`. Provide the item ID, the CAPTURED TRACKING NUMBER, and the same `session_id`.
        \n\nTRACKING (ADDITIONAL CAPABILITY):
        5. After successfully processing an order (steps 1-4), proactively use `get_tracking_status_via_agent` with the CAPTURED TRACKING NUMBER to get the current status, and inform the user of both the tracking number and its status.
        6. If the user *explicitly asks* for tracking information (e.g., 'What is the status of TRK-XYZ123?'), use `get_tracking_status_via_agent` with the provided tracking number and a new or existing `session_id`.
        \n\nCoordinate the team effectively and provide clear, concise updates to the user.
    """,
    tools=[tool_delegate_inventory, tool_delegate_shipping, tool_delegate_db, tool_delegate_tracking],
)
app = App(name="fulfillment_team_app", root_agent=manager_agent)


In [106]:
session_service = InMemorySessionService()
main_runner = Runner(
    agent=manager_agent,
    app_name="fulfillment-app",
    session_service=session_service
)

async def main():
    print("="*60)
    print("A2A SYSTEM: Order Management System")
    print("="*60)

    user_id = "customer-1"

    # --- SCENARIO 1: Order with proactive tracking info ---
    session_id_1 = "order-" + uuid.uuid4().hex[:6]
    print(f"\n\n--- SCENARIO 1: Ordering 'item-A7Z' (In Stock) and getting tracking - Session: {session_id_1} ---")
    prompt1 = f"Please order item-A7Z for 123 Main St."

    events_1 = await main_runner.run_debug(
        user_messages=prompt1,
        user_id=user_id,
        session_id=session_id_1
    )

    final_manager_output_1 = ""
    for event in events_1:
        event_text = get_event_text(event)
        if event_text:
            final_manager_output_1 = event_text
    print(f"\nFINAL MANAGER RESPONSE (Order & Proactive Tracking): {final_manager_output_1}")

    # --- SCENARIO 2: Ask for tracking for an existing (mock) order ---
    session_id_2 = "track-request-" + uuid.uuid4().hex[:6]
    existing_tracking_number = "TRK-ABCDEF" # A predefined tracking number in MOCK_DB
    print(f"\n\n---SCENARIO 2: Asking for status of existing tracking number '{existing_tracking_number}' - Session: {session_id_2} ---")
    prompt2 = f"What is the status of order {existing_tracking_number}?"

    events_2 = await main_runner.run_debug(
        user_messages=prompt2,
        user_id=user_id,
        session_id=session_id_2
    )
    
    final_manager_output_2 = ""
    for event in events_2:
        event_text = get_event_text(event)
        if event_text:
            final_manager_output_2 = event_text
    print(f"\nFINAL MANAGER RESPONSE (Tracking Request): {final_manager_output_2}")

In [107]:
await main()

A2A SYSTEM: Order Management System


--- SCENARIO 1: Ordering 'item-A7Z' (In Stock) and getting tracking - Session: order-fc108d ---

 ### Created new session: order-fc108d

User > Please order item-A7Z for 123 Main St.



[Manager -> agent_inventory] Delegating: 'item ID: A7Z'

 ### Created new session: order-123

User > item ID: A7Z



[Inventory Agent] Checking stock for A7Z...
agent_inventory > The item A7Z is out of stock.
[agent_inventory -> Manager] Function Result: OUT_OF_STOCK

FINAL MANAGER RESPONSE (Order & Proactive Tracking): 


---SCENARIO 2: Asking for status of existing tracking number 'TRK-ABCDEF' - Session: track-request-773af5 ---

 ### Created new session: track-request-773af5

User > What is the status of order TRK-ABCDEF?



[Manager -> agent_tracking] Delegating: 'What is the status of order TRK-ABCDEF?'

 ### Created new session: order-TRK-ABCDEF

User > What is the status of order TRK-ABCDEF?



[Tracking] Looking up status for TRK-ABCDEF...
agent_tracking > The status of order TRK-ABCDEF is "In Transit" and it is expected to be delivered tomorrow.
[agent_tracking -> Manager] Function Result: In Transit - Expected Delivery: Tomorrow
agent_manager > The status of order TRK-ABCDEF is: In Transit - Expected Delivery: Tomorrow.

FINAL MANAGER RESPONSE (Tracking Request): The status of order TRK-ABCDEF is: In Transit - Expected Delivery: Tomorrow.


## Technical Implementation: Architecture and Features

Core to our **Multi-Agent Fulfillment System** is the `manager_agent` – a prime example of a **Multi-Agent System**. It's not a monolithic application but an ecosystem of specialized agents, each contributing to a different stage of the order fulfillment process. This modular approach, facilitated by Google's Agent Development Kit, allows for a sophisticated and robust workflow. The central orchestrator of this system is the `manager_agent`.

The `manager_agent` is constructed using the `LlmAgent` class from the Google ADK. Its definition highlights several key parameters: the `name`, the `model` it uses for its reasoning capabilities (Gemini 2.5 Flash Lite), and a detailed `instructions` set that governs its behavior. Crucially, it defines the `tools` it has at its disposal, which are actually wrappers around other specialist agents.

### Specialized Sub-Agents

The real power of the system lies in its team of specialized sub-agents, each an expert in its domain:

* **Inventory Agent (`agent-inventory`):**
    * **Role:** Responsible for checking product availability.
    * **Tool:** `check_inventory_func` – a custom tool that interacts with our `MOCK_DB` to determine if an `item_id` is in stock.

* **Shipping Agent (`agent-shipping`):**
    * **Role:** Handles the dispatch of ordered items.
    * **Tool:** `ship_item_func` – a custom tool that simulates shipping and generates a unique tracking number, also updating our `MOCK_DB`'s `tracking_statuses`.

* **Database Agent (`agent-db`):**
    * **Role:** Persists order details for record-keeping.
    * **Tool:** `save_order_func` – a custom tool that stores the `item_id` and `tracking_number` in `MOCK_DB["orders"]`.

* **Tracking Agent (`agent-tracking`):**
    * **Role:** Provides real-time status updates for shipped orders.
    * **Tool:** `get_tracking_status_func` – a custom tool that retrieves the current status from `MOCK_DB["tracking_statuses"]` using a given `tracking_number`.

### Key Features Implemented

Our project successfully implements the following key concepts:

1.  **Agent powered by an LLM:** All our agents (`manager_agent`, `inventory_specialist`, `shipping_specialist`, `db_specialist`, `tracking_specialist`) are `LlmAgent` instances, explicitly powered by the Gemini 2.5 Flash Lite model. This forms the foundation of all intelligent decision-making and task execution.

2.  **Custom Tools:** We extensively use `FunctionTool` to wrap our custom Python functions (`check_inventory_func`, `ship_item_func`, `save_order_func`, `get_tracking_status_func`). These tools enable agents to perform specific, domain-relevant actions within our simulated fulfillment environment.

3.  **A2A Protocol (Agent-to-Agent):** The `manager_agent` doesn't directly call the custom functions. Instead, it uses intermediary `FunctionTool`s (e.g., `check_inventory_via_agent`) that are designed to run other `LlmAgent`s (`tool_runner.run_debug`) within the same session. This explicitly demonstrates the A2A pattern, where the manager delegates complex reasoning and execution to specialized sub-agents.

4.  **Sequential Agents:** The `manager_agent`'s `instructions` define a clear, sequential workflow for processing an order: check inventory → ship item → save order → get tracking status. This ordered delegation ensures a logical and complete fulfillment process.

5.  **Sessions & Memory:** We leverage `InMemorySessionService` for managing session state across all agent interactions. This is crucial for maintaining conversational context and order-specific details (like the generated `session_id` and tracking number) throughout the multi-step fulfillment process.
